<hr>
# [cknowledge.org](http://cknowledge.org): Community-driven benchmarking and optimization of computing systems - from classical to quantum
<hr>

[Quantum Computing](https://github.com/ctuning/ck-quantum/wiki)
* [CK-QISKit](https://github.com/ctuning/ck-qiskit) (IBM)
* [CK-Rigetti](https://github.com/ctuning/ck-rigetti)
* [CK-ProjectQ](https://github.com/ctuning/ck-projectq)

[Artificial Intelligence and Machine Learning](http://cknowledge.org/ai)
* [Reproducible Quality-Efficient Systems Tournaments](http://cknowledge.org/request) ([ReQuEST initiative](http://cknowledge.org/request.html#organizers))
* [AI artifacts](http://cknowledge.org/ai-artifacts) (cTuning foundation)
* [Android app](https://play.google.com/store/apps/details?id=openscience.crowdsource.video.experiments) (dividiti)
* [Desktop app](https://github.com/dividiti/ck-crowdsource-dnn-optimization) (dividiti)
* [CK-Caffe](https://github.com/dividiti/ck-caffe) (Berkeley)
* [CK-Caffe2](https://github.com/ctuning/ck-caffe2) (Facebook)
* [CK-CNTK](https://github.com/ctuning/ck-cntk) (Microsoft)
* [CK-KaNN](https://github.com/dividiti/ck-kann) (Kalray)
* [CK-MVNC](https://github.com/ctuning/ck-mvnc) (Movidius / Intel)
* [CK-MXNet](https://github.com/ctuning/ck-mxnet) (Apache)
* [CK-NNTest](https://github.com/ctuning/ck-nntest) (cTuning foundation)
* [CK-TensorFlow](https://github.com/ctuning/ck-tensorflow) (Google)
* [CK-TensorRT](https://github.com/dividiti/ck-tensorrt) (NVIDIA)
* etc.

# Variational Quantum Eigensolver - Hackaton

## Table of Contents

1. [Overview](#overview)
1. [Running experiments](#run)
1. [Sample experimental data](#data) [for developers]
1. [Data wrangling code](#code) [for developers]

<a id="overview"></a>
## Overview

This Jupyter Notebook studies the performance of [Variational-Quantum-Eigensolver](http://grove-docs.readthedocs.io/en/latest/vqe.html) (VQE) on [Rigetti](https://rigetti.com/)'s [Quantum Virtual Machine](http://pyquil.readthedocs.io/en/latest/qvm.html) (QVM), [Quantum Processing Unit](http://pyquil.readthedocs.io/en/latest/qpu.html) (QPU) and [IBM](https://ibm.com/)'s [QISKit](https://qiskit.org/) simulators.

<a id="run"></a>
## Run experiments

### Rigetti

```
$ ck benchmark program:rigetti-vqe --repetitions=3 \
  --record --record_repo=local --record_uoa=<email>[-<plaform>] \
  --tags=qck,hackathon-2018_06_15,<email>,<platform>,<minimizer_method> \
  --env.RIGETTI_QUANTUM_DEVICE=<platform> \
  --env.VQE_MINIMIZER_METHOD=<minimizer_method> \
  --env.VQE_SAMPLE_SIZE=<sample_number> \
  --env.VQE_MAX_ITERATIONS=<max_iterations>
```
where:
- `email`: a valid email address;
- `platform`: `8Q-Agave` or `QVM`;
- `minimizer_method`: `my_melder_nead` or `my_cobyla` or `my_minimizer` (as defined in [optimizers.py](https://github.com/ctuning/ck-quantum/blob/master/package/tool-hackathon/hackathon-src/hackathon/optimizers.py) installed under e.g. `$CK_TOOLS/hackathon-1.0-linux-64/lib/hackathon`);
- `sample_size`: e.g. `100` (or another resolution);
- `max_iterations`: e.g. `80` (or another cut-off point);

<a id="data"></a>
## Get sample experimental data

The sample experimental data can be downloaded and registered with CK as follows:

```
$ wget https://www.dropbox.com/s/fvichboqnjcxc8v/ck-quantum-hackathon-sample.zip
$ ck add repo --zip=ck-quantum-hackathon-sample.zip
```

In [ ]:
repo_uoa = 'ck-quantum-hackathon-sample'
!ck list $repo_uoa:experiment:* --print_full | sort

<a id="code"></a>
## Data wrangling code

**NB:** Please ignore this section if you are not interested in re-running or modifying this notebook.

### Includes

#### Standard

In [ ]:
import os
import sys
import json
import re

#### Scientific

If some of the scientific packages are missing, please install them using:
```
# pip install jupyter pandas numpy matplotlib
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 16
default_figheight = 8
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

#### Collective Knowledge

If CK is not installed, please install it using:
```
# pip install ck
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

### Access experimental data

In [ ]:
def get_experimental_results(repo_uoa, tags='qck', module_uoa='experiment'):
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']
    
    dfs = []
    for experiment in experiments:
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
        if r['return']>0:
            print('Error: %s' % r['error'])
            exit(1)
        tags = r['dict']['tags']
        
        skip = False
        # TODO: will break for when QISKit backends are used.
#        if tags not in ['qvm', '8q-agave']: skip = True
        if skip:
            print('[Warning] Skipping experiment with bad tags:')
            print(tags)
            continue

        # For each point.    
        for point in r['points']:
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']
            num_repetitions = len(characteristics_list)
            data = [
                {
                    # features
                    'platform': characteristics['run'].get('vqe_input', {}).get('q_device_name', 'unknown').lower(),
                    # choices
                    'minimizer_method': characteristics['run'].get('vqe_input', {}).get('minimizer_method', 'n/a'),
                    'minimizer_options': characteristics['run'].get('vqe_input', {}).get('minimizer_options', {'maxfev':-1}),
                    'sample_number': characteristics['run'].get('vqe_input', {}).get('sample_number','n/a'),
                    # statistical repetition
                    'repetition_id': repetition_id,
                    # runtime characteristics
                    'report': characteristics['run'].get('report', {}),
                    'vqe_output': characteristics['run'].get('vqe_output', {}),
                }
                for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                if len(characteristics['run']) > 0
            ]
            for datum in data:
                datum['success'] = datum.get('vqe_output',{}).get('success',False)
                datum['nfev'] = np.int64(datum.get('vqe_output',{}).get('nfev',-1))
                datum['nit'] = np.int64(datum.get('vqe_output',{}).get('nit',-1))
                datum['fun'] = np.float64(datum.get('vqe_output',{}).get('fun',0))
                datum['total_seconds'] = np.float64(datum.get('report',{}).get('total_seconds',0))
                datum['total_q_seconds'] = np.float64(datum.get('report',{}).get('total_q_seconds',0))
                datum['total_q_shots'] = np.int64(datum.get('report',{}).get('total_q_shots',0))
                datum['max_iterations'] = list(datum.get('minimizer_options',{'maxfev':-1}).values())[0]
            index = [
                'platform', 'minimizer_method', 'sample_number', 'max_iterations', 'repetition_id'
            ]
            # Construct a DataFrame.
            df = pd.DataFrame(data)
            df = df.set_index(index)
            # Append to the list of similarly constructed DataFrames.
            dfs.append(df)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs)
        result.sort_index(ascending=True, inplace=True)
    else:
        # Construct a dummy DataFrame the success status of which can be safely checked.
        result = pd.DataFrame(columns=['success'])
    return result

### Plot experimental data

In [ ]:
def plot(df, platform_set=None, minimizer_method_set=None, sample_number_set=None, max_iterations_set=None,
         markersize_divisor=20,
         xmin=0.0, xmax=85.01, xstep=5.00,
         ymin=-3.0, ymax=-0.49, ystep=0.25,
         figsize=(18,9), dpi=200, legend_loc='lower right'):
    
    platform_set = platform_set or df.index.get_level_values(level='platform').unique()
    minimizer_method_set = minimizer_method_set or df.index.get_level_values(level='minimizer_method').unique()
    sample_number_set = sample_number_set or df.index.get_level_values(level='sample_number').unique()
    max_iterations_set = max_iterations_set or df.index.get_level_values(level='max_iterations').unique()

    # Options.
    minimizer_method_to_color = {
        'my_cobyla' : 'orange',
        'my_nelder_mead' : 'green',
        'my_minimizer' : 'blue'
    }
    platform_to_marker = {
        '8q-agave' : '8', # octagon
        'qvm' : 's',      # square       
        'local_qasm_simulator' : 'p' # pentagon
    }
    
    last_marker_size = 10
    last_marker_color = 'black'
    last_marker_success_true = '^'
    last_marker_success_false = 'v'

    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.gca()
    for index, row in df.iterrows():
        (platform, minimizer_method, sample_number, max_iterations, repetition_id) = index
        if platform not in platform_set: continue
        if sample_number not in sample_number_set: continue
        if minimizer_method not in minimizer_method_set: continue
        energies = [ iteration['energy'] for iteration in row['report']['iterations'] ]
        marker=platform_to_marker[platform]
        markersize=sample_number/markersize_divisor
        color=minimizer_method_to_color[minimizer_method]
        markerfacecolor=color
        linestyle='-'
        ax.plot(range(len(energies)), energies, marker=marker, color=color, linestyle=linestyle,
                markerfacecolor=markerfacecolor, markersize=markersize)
        # Mark last function evaluation.
        last_marker = last_marker_success_true if row['success'] else last_marker_success_false
        ax.plot(row['nfev']-1, energies[-1], color=last_marker_color, marker=last_marker, markersize=last_marker_size)

    # Vertical lines for max_iterations.
    for max_iterations in max_iterations_set:
        plt.axvline(x=max_iterations, color='black')
    # Grid.
    plt.grid()
    # Title.
    title = 'Variational Quantum Eigensolver (VQE)'
    ax.set_title(title)
    # X axis.
    xlabel='Function evaluation'
    ax.set_xlabel(xlabel)
    ax.set_xlim(xmin, xmax)
    ax.set_xticks(np.arange(xmin, xmax, xstep))
    # Y axis.
    ylabel='Energy'
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin, ymax)
    ax.set_yticks(np.arange(ymin, ymax, ystep))
    # Legend. https://matplotlib.org/users/legend_guide.html
    handles = [
        mp.lines.Line2D([], [], label='platform="%s",minimizer_method="%s"' % (p,m), color=minimizer_method_to_color[m],
                        marker=platform_to_marker[p], markersize=last_marker_size)
        for p in sorted(platform_set)
        for m in sorted(minimizer_method_set)
    ]
    plt.legend(handles=handles, title='platform,minimizer_method', loc=legend_loc)
    # Save figure.
#    plt.savefig('vqe.energy.png')

In [ ]:
def plot_metric(df, metric='total_q_seconds'):
    df.columns.name='metric'
    # "df.index.names[:-1]" means reduce along 'repetition_id' (statistical variation).
    df_mean = df[[metric]].groupby(level=df.index.names[:-1]).mean().unstack('platform')
    df_std = df[[metric]].groupby(level=df.index.names[:-1]).std().unstack('platform')
    ax = df_mean.plot(kind='bar', yerr=df_std, grid=True, legend=True, rot=45,
                      fontsize=default_fontsize, figsize=default_figsize, colormap=default_colormap)

<a id="analysis"></a>
## Analysis

In [ ]:
df = get_experimental_results(repo_uoa=repo_uoa)
display_in_full(df)

### Plot convergence

In [ ]:
plot(df)

In [ ]:
plot(df, sample_number_set=[500])

In [ ]:
plot(df, minimizer_method_set=['my_cobyla'], sample_number_set=[5000], markersize_divisor=500,
     xmax=30+0.01, xstep=1, ymin=-1.79, ymax=-1.09+0.01, ystep=0.05, legend_loc='lower left')

In [ ]:
plot(df, minimizer_method_set=['my_cobyla'], sample_number_set=[50], markersize_divisor=5,
     xmin=5, xmax=32+0.01, xstep=1, ymin=-2.79, ymax=-1.79+0.01, ystep=0.05, legend_loc='lower left')

### Plot execution metrics

In [ ]:
plot_metric(df)

In [ ]:
plot_metric(df, metric='total_q_shots')